# Эта тетрадка с помощью правил элементарным образом размечает, есть ли определение в предложении или нет.

sent_index | text_sent | has_def
- | - | -
1 | текст предложения без определения |0
2 |текст предложения с определением |1

In [1]:
import pandas as pd

In [2]:
! pip install natasha

     |████████████████████████████████| 34.4 MB 147 kB/s 
     |████████████████████████████████| 41 kB 114 kB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 8.2 MB 40.2 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=42bc1a1a4de955e0882ba5b77a210993d955de5b345b11e51c606681a0a4a88e
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()

In [4]:
# открываем дф
df = pd.read_csv('sents_Wiki_tagged.csv', sep=';', dtype={'sent_index': str, 'sent_text': str, 'def': int})

In [5]:
df

,Unnamed: 0,sent_index,sent_text,def
0,0,0,"Лингви́стика (от лат. lingua «язык»), языкозна...",1
1,1,12,"Как дисциплина, имеющая ряд принципиальных чер...",1
2,2,13,"Лингвистами часто становятся люди, совмещающие...",1
3,3,15,Лингвистика в широком смысле слова (познание я...,1
4,4,17,"Она бывает: нормативной: указывающей, как «на...",0
...,...,...,...,...
921,921,24786,Каждая задача впоследствии оценивается определ...,0
922,922,24787,"Командное соревнование, за исключением первой ...",0
923,923,24788,Сданное решение оценивается жюри по определённ...,0
924,924,24789,"Официальный сайт со статистикой, условиями и р...",0


In [26]:
df['def_rules'] = 0
patterns = [' — ', 'то есть', 'называют', 'называется', 'называются','является', 'являются']
for index, row in df.iterrows():
    for pattern in patterns:
        if pattern in row['sent_text']:
            df['def_rules'][index] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
error_1_0 = 0
error_0_1 = 0
for index, row in df.iterrows():
    if row['def'] > row['def_rules']:
        error_1_0 +=1
    if row['def'] < row['def_rules']:
        error_0_1 +=1

In [28]:
error_1_0, error_0_1

(26, 114)

In [29]:
len(df)

926

### Смотрим качество

In [30]:
from sklearn.metrics import classification_report

In [31]:
gold = df['def']
pred = df['def_rules']
report = classification_report(gold, pred, target_names=['no term-def', 'term-def'])
print(report)

              precision    recall  f1-score   support

 no term-def       0.92      0.74      0.82       432
    term-def       0.80      0.95      0.87       494

    accuracy                           0.85       926
   macro avg       0.86      0.84      0.84       926
weighted avg       0.86      0.85      0.85       926

